# Making a new clemgame
*While written for prior versions of `clemcore`, reading the documentation on [how to add new games](howto_add_games.md) and how to [log events and build records](logging_and_scoring.md) is recommended, but not necessary.*

## Index
[Game concept](#game-concept)

[Clemgame components](#clemgame-components)

[Clemgame template and workspace setup](#clemgame-template-and-workspace-setup)

[Instances and resources basics](#instances-basics-and-game-resources)

[Player implementation](#player-implementation)

[GameMaster implementation](#gamemaster-implementation)

[GameBenchmark implementation](#gamebenchmark-implementation)

[Testing and refinement](#testing-and-refinement)

The core parts of a clemgame are separated into classes:
- **Player** class handles players of the game, holding their message history and making calls to a backend model to generate responses.
- **GameMaster** class handles the game loop, determining what each player gets prompted with when
- **GameScorer** class handles scoring based on episode records
- **GameBenchMark** class coordinates benchmark runs of the clemgame, initializing GameMaster with game instances and GameScorer with recorded episodes

# Game concept
The first step of creating a new clemgame is to come up with a game concept, or to find an existing dialogue game to adapt. In either case, it is important that the rules of the game can be implemented programmatically. While you can implement a clemgame with complex parsing, it is recommended to keep rules simple enough to understand benchmark results without learning code intricacies of the clemgame implementation.

## Example game concept
Let's implement a simple two-player game we will call `firstlast`.

The players should engage in a back-and-forth conversation about a predefined topic. Player A starts with an utterance whose first and last word must start with a predefined letter, say  `d`. Player B must then reply with an utterance whose first and last word must be the next one in the alphabet (here, an `e`). And so on, for `n` rounds (where each round comprises a single turn, with an utterance from A or B). If an utterance does not conform to these rules (i.e. it is incorrect), the players lose the game. We also define a move format rule: If an utterance does not start with `I SAY: ` (i.e., it is invalid), the game is immediately aborted. If all utterances up to turn `n` are valid and correct, the game is successful.

For instance, if the topic is `birds`, the initial letter is `h` and the number of ~~turns~~ rounds is 2, this would be a successful game:

- Player A: "Hi! I love birds, but it's hard to identify them. I need help." (h: hi / help)
- Player B: "I know what you mean. I can try to help, please describe it." (i: I/ it)
- Player A: "Just a moment... Ok, it's blue but looks like an Eurasian jay." (j: just / jay)
- Player B: "Kick in more details, otherwise I don't know." (k: kick / know)

In each round, we need to check two aspects:
- Does the utterance have a valid form fit to be parsed? Failing to meet this leads to the game being aborted. (Move format rule.)
- Does the utterance fulfil the game rules for a successful round? Failing to meet this leads to game being lost. (Game rules.)

To make things simple, in this example we will check only these conditions:
- Validity: Does the utterance start with `I SAY:`?
- Correctness: Do the first and last words begin with the same, correct letter at play?


# Clemgame components
A clemgame needs at least the following components:

1. [Game resources](#instances-basics-and-game-resources): All data, prompt templates and other files that are necessary to create instances of a game and to group these instances into experiments.
2. [Instances](#instances-basics-and-game-resources): A JSON file containing the configuration of each instance of this game, grouped into experiments. This must be done by a script named `instancegenerator.py`, with a class that inherits from `GameInstanceGenerator`.
3. [Players](#player-basics): A script that defines the programmatic behaviour and any other attributes of a player, inheriting from the `Player` base class. (This can be implemented in a file named `players.py`.)
4. [Game Master](#gamemaster-implementation): A script that controls and enforces the defined move and game rules and dynamics, inheriting from the `GameMaster` base class. This must be implemented in the `master.py` file.
5. [Game Benchmark](#gamebenchmark-implementation): A class that realises the game's running behavior for benchmarking, inheriting from `GameBenchmark`. This can also live in the file `master.py`.
6. [Game registry](#game-registry) entry: A JSON-format specification of the clemgame required by the `clemcore` framework to locate the clemgame's files.

Let's walk through the implementation step by step. We'll write the contents of `instancegenerator.py`, `master.py` and `players.py`, that you should save as files to run the game.

**Note**: For the mandatory methods, always check the parent class documentation to be sure about the required and optional arguments.

# Clemgame template and workspace setup
The most convenient way to create a clemgame is to start with the template repository. It contains a minimal clemgame implementation as a base to work from, found in the `empty_template` directory.

To start, follow these steps:
1. Clone the clemgame template repository from https://github.com/clembench/clemgame-template . The directory you clone it to will be the workspace directory.
2. Set up a python virtual environment with `clemcore`, as described in the [template readme](https://github.com/clembench/clemgame-template/blob/main/README.md).
3. Inspect the template's files and structure.
4. Create a directory for your clemgame: Copy the `empty_template` directory and rename it. For the example clemgame below, name it `firstlast`.

The example implementation below is based on this template.

# Instances basics and game resources
Instances of the `taboo` clemgame are located under `clembench/taboo/in` in the `instances.json` file. All clemgames must follow this directory structure, with the clemgame's base directory containing a subdirectory named `in` which contains the `instances.json` file.

The `instances.json` file contains a JSON object with the single key `experiments`, which holds a list of JSON objects, each corresponding to an experiment. An experiment holds clemgame parameters that apply for all its instances, and is used to evaluate and compare scores between different variants of a clemgame. The keys `name`, to identify different experiments, and `game_instances` are mandatory for all experiment objects.

An experiment object's `game_instances` key holds a list of the experiment's instances, each containing different data that is needed by the Game Master (explained below) to play an individual episode of the clemgame.
## Game resources
Resources - located in the `resources` subdirectory - are files that are accessed for instance generation or by the Game Master running the clemgame.

Conventionally, initial prompts that explain the game and its rules, and contain the starting state for an episode, are stored as `.template` files in `resources/initial_prompts`. Clemcore provides convenient methods to load these `.template` files (see below).

As you can see in the `taboo` files, initial prompt templates usually contain placeholder strings like `$TARGET_WORD$` that are replaced with instance values to create individual initial prompts for each instance.

## Example clemgame instances and resources
(In this example, defining an episode requires instantiating the initial prompts and 3 additional parameters: The topic, the letter for the first player and the number of turns for that game play.)

In this example, an instance defines which initial prompts to use and 3 additional parameters: The topic, the letter for the first player and the number of rounds that need to be successfully played.

### Defining prompts with game rules

The players need to be instructed on what the rules of the game are, possibly with some examples, at the beginning of the game. For that, we must define the initial prompts passed to player A and player B.

In the template text, we can define variables that will later be filled with our chosen values (here: topic, first letter, number of rounds). The prompts need to be adjusted for player A and B according to their roles. For example:

Player A:
```"Let's play a game. You must have a conversation about $topic with your partner. Your first turn must start and end with words that begin with the letter $letter. The reply of your partner must be similar, with the letter that comes after $letter in the alphabet. Then it's your turn again with the next letter, and so on. You'll do it for $n_turns turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."```

Player B:
```"Let's play a game. You must have a conversation about $topic with your partner. Their first turn must start and end with words that begin with the letter $letter. Your reply must be similar, with the letter that comes after $letter in the alphabet. Then it's their turn again with the next letter, and so on. You'll do it for $n_turns turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."```
NOTE: These templates are assuming the use of `string.Template` to fill the placeholders (starting with `$`, like `$n_turns`).

You can later refine these initial prompts based on how models handle them. Save the initial prompts as plain texts using `.template` as an extension in the `initial_prompts` directory. You can save many templates if you wish to test different prompts, and read each of them when you generate game instances (see below). Note that this template is just an example and has not been tested. (Decide what amount of prompt engineering you will do at this step. Once you are satisfied with your preliminary results (i.e. the model can process the instructions well enough for your purposes), save to file...)

### Additional resources
Everything else needed to create instances for the game or to be accessed by the game master should be saved into the `firstlast/resources` directory as well.

Let's create a `topics.txt` file with a list of topics that we can later sample from to create our instances. The following cell will do so:

In [ ]:
topics = ['dogs', 'cats', 'birds', 'trees']

with open('resources/topics.txt', 'w') as file:
    for topic in topics:
        file.write(topic + '\n')

You can also write the topics manually using any text editor or IDE. Intended `topics.txt` content:
```
dogs
cats
birds
trees
```
### Creating game instances
Create a Python script called `instancegenerator.py` in `firstlast/`. Running this file will create a `JSON` file in `firstlast/in/`, called `instances.json`. The clemcore framework provides base classes to organise this. All we need to do is write a class that inherits from the `GameInstanceGenerator` class and write its `on_generate` method according to our needs. Then, in the main call, instantiate this class and call its `.generate()` method.

In the `on_generate` method, we define experiments and then define instances of each experiment. An instance is a configuration of one specific game play and an experiment is a set of related instances. We can define what is an experiment and what is an instance depending on what dimensions we want to evaluate later.

For our game, let's define an experiment as a set of instances about the same topic. To define an instance in an experiment, we define the initial letter, the initial prompts and the number of turns. This is useful if we wish to evaluate the performance of LLMs on variations of the same topic. Another possibility would be to define experiment as a set of instances with the same number of turns, and then each instance could be about different topics. That's our choice.

Running this will automatically create a `JSON` file with a key `experiments`, which is a list of experiments. Each element has a name and a list of `game_instances`. A game instance should have at least a `game_id` assigning an index to that instance and other keys and values that are necessary to play the game. Here, the initial prompts can have their slots already filled with the instance's values (or we can leave that for the `setup` method of the game master).

Here is an example of the structure we need:

```JSON
{
    "experiments": [
        {
            "name": "NAME_1",
            "game_instances": [
                {
                    "game_id": 0,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
                {
                    "game_id": 1,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
            ]
        },
        {
            "name": "NAME_2",
            "game_instances": [
                {
                    "game_id": 0,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
                {
                    "game_id": 1,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
            ]
        },
    ]
}
```

***Note***: The `instances.json` file should contain everything that the game master needs to set up the configuration to play the game! We can add as many keys and values and we need.

Here is an example:

In [ ]:
# save the contents of this cell as games/firstlast/instancegenerator.py
import os
import random
import string
import logging

from clemcore.clemgame import GameInstanceGenerator

# initialize logging:
logger = logging.getLogger(__name__)

# set the name of the game in the script, as you named the directory
# this name will be used everywhere, including in the table of results
GAME_NAME = 'firstlast'
# we will create 10 instances for each experiment; vary this as you wish
N_INSTANCES = 10
# if the generation involves randomness, remember to set a random seed
SEED = 123

class FirstLastGameInstanceGenerator(GameInstanceGenerator):
    def __init__(self):
        # always do this to initialise GameInstanceGenerator
        super().__init__(os.path.dirname(__file__))

    # define on_generate, a mandatory method
    def on_generate(self):
        # get the list of topics, which will be our experiments
        topics = self.load_file('resources/topics.txt').strip('\n').split('\n')
        # get the prompts for player a and player b
        # we'll keep the prompts fixed in all instances, replacing only the
        # necessary slots (but you can do it differently)
        prompt_a = self.load_template('resources/initial_prompts/initial_prompt_a')
        prompt_b = self.load_template('resources/initial_prompts/initial_prompt_b')

        # building the file, one experiment at a time
        for topic in topics:
            # create an experiment (for us, named after a topic)
            experiment = self.add_experiment(topic)
            # build N_INSTANCES instances for each experiment
            for game_id in range(N_INSTANCES):
                # set the parameters
                # here we do it randomly, but that can also be read from a file
                # one of the first 5 letters in the alphabet
                letter = random.choice(string.ascii_lowercase[:5])
                # up to 8 turns, so that we don't run out of letters
                n_turns = random.randint(3, 8)
                # create a game instance, using a game_id counter/index
                instance = self.add_game_instance(experiment, game_id)
                # populate the game instance with its parameters
                instance['first_letter'] = letter
                instance['n_turns'] = n_turns
                instance['prompt_player_a'] = self.create_prompt(
                    topic, prompt_a, letter, n_turns)
                instance['prompt_player_b'] = self.create_prompt(
                    topic, prompt_b, letter, n_turns)

    # an additional method, specific for our example
    def create_prompt(self,
                      topic: str,
                      prompt: str,
                      letter: str,
                      n_turns: int) -> str:
        """Replace a prompt template with slot values."""
        text = string.Template(prompt).substitute(topic=topic, letter=letter,
                                                  n_turns=n_turns)
        return text


if __name__ == '__main__':
    random.seed(SEED)
    # always call this, which will actually generate and save the JSON file
    FirstLastGameInstanceGenerator().generate()


Summary:

- Write a class that inherits from ```GameInstanceGenerator```.
- You must implement the ```on_generate``` method, which should call ```self.add_experiment()``` to add experiments and ```self.add_game_instance()``` to add instances. Populate the game instance with keys and values.
- ```GameInstanceGenerator``` has methods to load various files inside the game directory, for example ```self.load_template()``` and  ```self.load_file()```.
- In ```'__main__'```, call ```FirstLastGameInstanceGenerator().generate()```.
- Set a random seed if your generation relies on randomness; when you need new instances, change the random seed.

An ideal clemgame instance generation allows creating varied new sets of instances by simply running `instancegenerator.py` with a new seed value.

# Player implementation
Players are handled by child classes of the clemcore `Player` class, which holds the message history of a player and communicates with the model backend. A programmatic response method for testing is also commonly set up according to the clemgame.
## Defining the Player class
Create a python file called `player.py` in the workspace directory. (If your player class/es are very compact, you might define them in `master.py` instead, the creation of which is covered [below](#gamemaster-implementation). The 'empty' template has a minimal player definition in its `master.py` which you can extend for this purpose.)

In our game, the role of player A and B are symmetric, i.e. they behave the same way and have the same tasks and goals. So we can define one class and instantiate both players from it. If in your game players have different roles, then define two types of Player objects. The only method that we must implement is `_custom_response()`, which must define a programmatic behaviour for this player. The rest (getting and generating utterances via API calls to LLMs) is taken care of by the framework (we'll see below how to use it). Of course, you can add more methods that relate to the behaviour of the player in your game.

The programatic behaviour is useful in two cases: when the player is really a program (i.e. it sends only predefined messages, e.g. read from a file, not retrieved from an LLM agent) or for testing your program, using the `mock` setting that does not make API calls to any LLMs. For the first case, the argument `model_name` in the initialisation should be set to `"programmatic"`.

We also initialise a list to represent the dialogue history of this player. It will be incrementally built during the game play by appending new utterances to it.

In [ ]:
# save the contents of this cell as firstlast/players.py

import random
import logging
from string import ascii_lowercase as letters
from typing import List, Dict, Union

from clemcore.clemgame import Player
from clemcore.backends import Model

# initialize logging:
logger = logging.getLogger(__name__)

class Speaker(Player):
    def __init__(self, model: Model, letter: str, firstlast_player: str, name: str = None):
        # if the player is a program and you don't want to make API calls to
        # LLMS, use model='{"model_name": programmatic"}'
        super().__init__(model, name)
        self.player: str = firstlast_player
        self.initial_letter: str = letter

    # implement this method as you prefer, with these same arguments
    def _custom_response(self, context: Dict) -> str:
        """Return a mock message with the suitable letter and format.
        Args:
            context: The dialogue context to which the player should respond. Base class method, not used in this example.
        Returns:
            Mock message with the suitable letter and format.
        """
        # get the first letter of the content of the last message
        # messages is a list of dictionaries with messages in openai API format
        turn_idx = len(self._messages)  # will be 1 if only initial prompt message is in message history

        if turn_idx == 1 and self.player == 'A':
            letter = 'I SAY: ' + self.initial_letter
        else:
            previous_letter = self._messages[-1]['content'][7].lower()
            # introduce a small probability that the player fails
            letter = self._sample_letter(previous_letter)
        # return a string whose first and last tokens start with the next letter
        return f"{letter}xxx from {self.player}, turn {turn_idx} {letter.replace('I SAY: ', '')}xxx."

    # an additional method specific for this game
    # for testing, we want the utterances to be invalid or incorrect sometimes
    def _sample_letter(self, letter: str) -> str:
        """Randomly decide which letter to use in a custom response message."""
        prob = random.random()
        index = letters.index(letter)
        if prob < 0.05:
            # correct but invalid (no tag)
            return letters[index + 1]
        if prob < 0.1:
            # valid tag but wrong letter
            return 'I SAY: ' + letter
        # valid and correct
        return 'I SAY: ' + letters[index + 1]


Summary:

- Write a class that inherits from `Player`.
- Define its `_custom_response()` method, which implements the programmatic behaviour of the player (for testing, or because it is really a program) and returns a string.
- A `ModelSpec` defines the model to use, and the programmatic custom response is used by passing `model='{"model_name": programmatic"}'`.

# GameMaster implementation
The core clemgame functionality is implemented in its game master class, containing the play loop and handling any game-specific interaction. While a game master can be fully custom and based on just the minimal implementation in the clemcore `GameMaster` base class, the more extensive `DialogueGameMaster` base class inheriting from `GameMAster` is commonly used. `DialogueGameMaster` implements the game conversation loop described in the clembench papers, and only requires extension for game-specific behavior. The game master must be implemented in `master.py`, located in the root directory of a clemgame.
## Defining the GameMaster class
Open `master.py` in the `firstlast` directory. Remove the `class SomeGamePlayer(Player)` definition (for your own clemgame, you can rename this class and add it to it if your player class does not need much extra code).

We need define the game master, a class that inherits from `DialogueGameMaster` and implement certain required methods, mainly its `_on_setup()` method and play loop methods like `_does_game_proceed()`.

The metrics that every game must compute are listed at `clemcore/clemgame/metrics.py`, described in [log events and build records](logging_and_scoring.md) and in more detail in the paper's appendix. Note: You should **not** implement `METRIC_PLAYED` if you use the provided evaluation scripts, because this metric is inferred from `METRIC_ABORTED` there. Besides, any number of additional game-specific metrics can also be logged (see more below).

These are the mandatory methods. However, for readability, we will also write auxiliary methods.

**IMPORTANT**: The game master has to log ***every event*** that is relevant to reconstruct the interaction, build the transcript and evaluate the game.

The `DialogueGameMaster` is also a `GameResourceLocator`, which has special methods to access and write files in the game's local directory, and a `GameRecorder`, which knows how to log events. We'll see below how to use it.

The `GameMaster` and `DialogueGameMaster` base classes have a `GameResourceLocator` to access resources and a `GameRecorder` to be used by all involved class objects.
### Initialisation

The first step is to initialise the game master. The `__init__` method gets the experiment object and a list of `Model` objects (`Model` objects are given by the benchmark scripts and handled by clemcore backends).

Rename the `DialogueGameMaster` child class in `master.py` to `FirstLast`.

Change the docstring of the class to fit the game: `"""Implement mechanisms for playing FirstLast."""`.

The beginning of `master.py` should now look like this:
```python
import os.path
from typing import Dict, Tuple, List, Union
from string import ascii_lowercase as letters
import logging

import numpy as np

import clemcore.metrics as ms
from clemcore.clemgame import GameSpec, GameMaster, GameBenchmark, Player, DialogueGameMaster, GameScorer, \
    GameError, ParseError, RuleViolationError
from clemcore.backends import Model

# import the Speaker player class:
from player import Speaker

# initialize logging:
logger = logging.getLogger(__name__)

class FirstLast(DialogueGameMaster):
    """Implement mechanisms for playing FirstLast."""
    def __init__(self, game_name: str, game_path: str, experiment: Dict, player_models: List[Model]):
        super().__init__(game_name, game_path, experiment, player_models)

````

### Keeping records
`DialogueGameMaster` handles all default record keeping, but game-specific records need to be implemented using the `log_key()` method.

Note: see details about keeping records in [log events and build records](logging_and_scoring.md).
#### Default record keeping
All events that occur during the game, i.e. all actions by the game master and by the players, must be documented. This is done by the methods of `GameRecorder` of the `GameMaster`. The essential ones are:

- At the beginning of every turn, call `log_next_turn()` (a turn is one player response and its processing by the game master). This is already implemented in the `DialogueGameMaster` base class play loop.
- In the game setup, call `log_players()` in order to log the models that are playing this episode of the game. This is already implemented in the `DialogueGameMaster` base class setup.
- Use `log_event()` to log all types of actions with a `to` and `from_`.
- The `action` object passed to `log_event()` must contain at least a key `type` and a key `content`. The first can be `send message`, `get message`, `metadata`, `parse`, `error`, `invalid format` or any game-specific types. Content is the actual message to de displayed in the transcript.
- Use only the values 'Player 1', 'Player 2' or 'GM' for the `from_` and `to` arguments. Messages that the game master emits to itself should have 'GM' both in `from_` and `to`.
- All events that involve making an API call should pass an additional `call` argument to `log_event()` containing the actual and exact API input and output objects, for posterior inspection if necessary. The `Player` base class already implements this.
- Any other episode-level object needed for scoring or documentation can be logged with `log_key()`.
#### Record files
Every action that is logged gets saved into the episode's `interactions.json` file after it is played. This file is then used to build game transcripts and to compute evaluation scores. The episode's `requests.json` file contains the API calls, saved when `log_event()` is called with a `call` argument. If you use a list, make deep copies to guarantee that you are not logging an object that mutates.
#### Records and scoring
The `GameScorer`'s (see [here](#gamescorer-implementation)) `compute_scores()` method gets the `interactions.json` dictionary as argument, so every key and value that is necessary to compute scores should be logged into the interaction file.
### IMPORTANT: Inspecting the game records
During development, always check the generated `interactions.json` and `requests.json` to make sure that the API calls are passing the correct structure and that the records are being correctly saved.

`interactions.json` is built by the game master as a way to represent the actual interaction (with all its meta-events like parsing messages or checking game rules). This is used to create the transcripts, which are a user-friendly visualisation of the interaction. But remember that this does not reflect the actual API calls, this only reflects what the game master makes of the game!

[Testing and refinement](#testing-and-refinement) has an example of checking `interactions.json` records.

The actual prompts and responses from the model are saved into `requests.json`, when an action is logged with its corresponding prompt and response object (see below how to do it). This file will reflect what was actually passed to and from the LLM. Remember that LLMs do not keep a internal state, so every call to a model must contain its full dialogue history. Also remember that when there are two LLMs playing at once, each will have its own dialogue history, which may be different! That's why, for debugging purposes, only looking at `interactions.json` is not enough, because it may not reflect exactly what the LLMs consumed and output.

#### Logging framework level events
To log framework level events for debugging and benchmarking, the framework standard logger is defined and used: `logger = get_logger(__name__)`. To log these, use `logger.log(<log entry>)`. Everything logged this way is appended to `clembench.log`.
### Episode setup
The `_on_setup()` method gets all keys=values in the instance dictionary, as we defined above. Use this method to set up everything that is needed so that the game can be played.

We want to access certain game instance values easily, like the number of turns need to successfully finish a game of `firstlast`. In the template `_on_setup()` method definition, assign the game instance value to `self.n_turns`: `self.n_turns = game_instance['n_turns']`

We then instantiate both players (including adding the initial prompts to their message history):
Replace
```python
self.some_player = SomeGamePlayer(self.player_models[0])
# with
self.player_a = Speaker(self.player_models[0], 'A', game_instance['first_letter'])
self.player_b = Speaker(self.player_models[1], 'B', game_instance['first_letter'])
```

The two players need to be added to allow iteration. We also pass them their respective initial prompts here:
Replace
```python
self.add_player(self.some_player, initial_context=game_instance['initial_prompt'])
# with
self.add_player(self.player_a, initial_context=game_instance['prompt_player_a'])
self.add_player(self.player_b, initial_prompt=game_instance['prompt_player_b'])
```
Note that the first player assignment uses the argument `initial_context`, while the second one uses `initial_prompt` - this is required at player assignment to properly establish the first player's context, otherwise its message history would be incomplete.

Next we assign the `current_turn` attribute, to keep track of the number of turns. (The `DialogueGameMaster` parent class already has the attribute `current_round`, but we want to track individual turns for `firstlast`.) We assign the first letter to the attribute `current_letter` and log it to be accessible for scoring later:
```python
# initialise game variables:
self.current_turn: int = 0
self.current_letter: str = game_instance['first_letter']
# log additional key that will be relevant for evaluation:
self.log_key('n_turns', game_instance['n_turns'])
```

To keep track of correctness of the latest response (`correct_response`), turn-level success scores (`turn_scores`) and the number of correctly completed turns (`complete_turns`), we assign corresponding attributes:
```python
self.correct_response = False
self.turn_scores = [0] * (self.n_turns + 1)
self.complete_turns: int = 0
```

Lastly, we assign attributes to keep track of (in)valid requests (`request_count, parsed_request_count, violated_request_count`) and if the game was aborted or lost:
```python
# initialise common metrics:
self.request_count: int = 0
self.parsed_request_count: int = 0
self.violated_request_count: int = 0

# initialise attributes that will be used for the evaluation scores
self.aborted: bool = False
self.lose: bool = False
```
**IMPORTANT:** These values are mandatory clemcore scores and MUST be implemented and recorded!

The `FirstLast` `_on_setup()` method should now look like this:
```python
    def _on_setup(self, **game_instance) -> None:
        """
        Set up the episode (mandatory).
        Args:
            game_instance: The game instance dict.
        """
        self.game_instance: dict = game_instance

        self.n_turns: int = game_instance['n_turns']

        # instantiate both players:
        self.player_a = Speaker(self.player_models[0], 'A', game_instance['first_letter'])
        self.player_b = Speaker(self.player_models[1], 'B', game_instance['first_letter'])

        # add players, including assigning their initial prompts:
        self.add_player(self.player_a, initial_context=game_instance['prompt_player_a'])
        self.add_player(self.player_b, initial_prompt=game_instance['prompt_player_b'])

        # initialise game variables:
        self.current_turn: int = 0
        self.current_letter: str = game_instance['first_letter']
        # log any additional keys that will be relevant for evaluation
        self.log_key('n_turns', game_instance['n_turns'])

        self.correct_response = False
        self.turn_scores = [0] * (self.n_turns + 1)
        self.complete_turns: int = 0

        # initialise common metrics:
        self.request_count: int = 0
        self.parsed_request_count: int = 0
        self.violated_request_count: int = 0

        # initialise attributes that will be used for the evaluation scores
        self.aborted: bool = False
        self.lose: bool = False
```
Summary:
- The setup must define players and log other game-specific keys.

### Playing the game
The `DialogueGameMaster` base class comes with a play loop already implemented. It has a number of hook methods that are called inside of this play loop, and game specifics are to be implemented within these methods.

This is the `play()` method of `DialogueGameMaster`:
```python
    def play(self) -> None:
        """Main play loop method.
        This method is called to run the game for benchmarking.
        """
        done = False
        while not done:
            # get the current context message for the current player, set by set_context_for():
            context = self.get_context_for(self.current_player)
            # generate/get response from the player based on their message history and the passed context message:
            response = self.current_player(context)
            # pass the player response to the step() method for processing and determining if play continues:
            done, _ = self.process_turn(response)
```
The `process_turn()` method makes method calls in this sequence:
1. **\_parse_response(self.current_player, response)**: Decide if a response follows the move format rule, should be modified and apply modifications.
2. **\_advance_game(self.current_player, parsed_response)**: Method executed after a player response has been parsed. Checks for game rule adherence and advances the game state.
3. **\_on_parse_error()**: Method executed if the response could not be parsed, usually due to not following the move format rule.
4. **\_on_game_error()**: Method executed if the response is not following further game rules.
5. **compute_turn_score(response, context)**: Calculate a turn-level score for this player response.
6. get_turn_feedback(response, context): Create textual feedback to the player response.
7. _should_pass_turn(): Determine if a player is done for the current round.
8. _next_player(): The game master passes the turn to the next player in the player list (order as added).
9. _start_next_round(): Start next round when we cycled through the whole list i.e. it is again the first player's turn.
10. **\_does_game_proceed()**: Check if game should proceed.
11. _on_before_round(): Executed in the play loop before a new round of gameplay starts.
12. _on_after_round(): Executed in the play loop after a round of gameplay finishes i.e. _start_next_round() resolves to True.
13. **_on_after_game()**: Executed once at the end, after exiting the play loop.

While all listed methods can be adapted for your game, we will implement adapted versions of the bolded methods. Implementation of the methods `_on_setup()`, `compute_turn_score()`, `compute_episode_score()`, `_advance_game()` and `_does_game_proceed()` is mandatory.

#### _parse_response()
The `_parse_response()` method checks for move format rule adherence and processes the response further. Since every response comes from a backend request, we increment `request_count`. We also add the response of the current player as the context message for the next player's round. For `firstlast`, we have a single move format rule: Player responses must start with `I SAY: `. If this rule is not followed `ParseError` is raised and the episode is aborted ([here](#_on_parse_error())). If the rule was followed, we log an event for transcripts and return the first and last word of the response (removing `I SAY:`). Replace the template method with the following:
```python
    def _parse_response(self, player: Player, response: str) -> Tuple[str, str]:
        """
        Add the response to the other player's message history and check if the response follows the move format rule,
        then split the response and return the first and last word.
        Args:
            player: The player that produced the response.
            response: The response string.
        Returns:
            Tuple of the first and last word of the response.
        Raises:
            ParseError: If the response is missing 'I SAY: '.
        """
        # increase the number of API requests:
        self.request_count += 1

        if player == self.player_a:
            self.set_context_for(self.player_b, response)
        if player == self.player_b:
            self.set_context_for(self.player_a, response)

        # check for move format tag:
        if not response.startswith("I SAY: "):
            raise ParseError()

        # increase the counter of requests that conform to form rules
        self.parsed_request_count += 1
        # log the event that the string was valid (no strange characters)
        action = {'type': 'metadata', 'content': 'move format followed'}
        self.log_event(from_='GM', to='GM', action=action)

        # remove the move format tag and split on whitespace:
        words = response[7:].split()
        return words[0].lower(), words[-1].lower()
```

#### _on_parse_error()
This method aborts the game if the move format rule was not followed. It is called if a `ParseError` is raised during the execution of `_parse_response()` and `_advance_game()`. We also increment the `violated_request_count` and log that the response didn't follow the required format for transcripts. Replace the template method definition with the following:
```python
    def _on_parse_error(self, error: GameError):
        """Abort the game due to failed parsing."""
        # set the game to be aborted:
        self.aborted = True
        # increase the counter of requests that violate the move format rule:
        self.violated_request_count += 1
        # log the abortion event:
        action = {'type': 'missing tag', 'content': 'abort'}
        self.log_event(from_='GM', to='GM', action=action)
```

#### _advance_game()
The `_advance_game()` checks if the player's response conforms to the game's other rules. If the letter rule isn't followed, the episode is lost. If the rules are followed, we log an event for trsnacripts and set this turn's score to 1. Then the method advances the game state if the response follows the rules, setting relevant attributes. We increment `current_turn` and set the next letter in the alphabet to be the new `current_letter`. Replace the template method definition with the following:
```python
    def _advance_game(self, player: Player, parsed_response: Tuple[str, str]):
        """
        Check if the parsed response follows the game rules. Then advance the game state, preparing the next player's turn.
        Args:
            player: The current player.
            parsed_response: The parsed response, a tuple of the first and last word of the response.
        """
        first_word_correct_letter = parsed_response[0][0] == self.current_letter  # True if the first letter of the first word in the response is correct
        last_word_correct_letter = parsed_response[0][0] == parsed_response[1][0]  # True if the first letters of the first and last word match
        self.correct_response = first_word_correct_letter and last_word_correct_letter
        if not self.correct_response:
            raise RuleViolationError(f'{parsed_response[0]}/{parsed_response[1]} violates rules')  # RuleViolationError inherits from GameError
        else:
            # log the fact that the answer was correct:
            action = {'type': 'valid response',
                      'content': f'{parsed_response[0]}/{parsed_response[1]} conforms to rules'}
            self.log_event(from_='GM', to='GM', action=action)
            # set the current turn's score to 1:
            self.turn_scores[self.current_turn] = 1
        # increment current turn:
        self.current_turn += 1
        # increment complete turns:
        self.complete_turns += 1
        # update the letter being played:
        current_index = letters.index(self.current_letter)
        self.current_letter = letters[current_index + 1]
```

#### _on_game_error()
This method declares the game lost if the game rules were not followed. It is called if a `GameError` is raised during the execution of `_parse_response()` and `_advance_game()`. We also log that the response didn't follow the rules for transcripts. Replace the template method definition with the following:
```python
    def _on_game_error(self, error: GameError):
        """Lose the game due to violated rules."""
        self.lose = True
        # log the fact that the game is now lost:
        action = {'type': 'rule violation',
                  'content': error.reason}
        self.log_event(from_='GM', to='GM', action=action)
```

#### compute_turn_score()
This method assigns a score for the response for the playpen reinforcement learning part of clemcore. We simply use the prior turn score for this (since we already incremented the turn in `_advance_game()`). Replace the template method definition with the following:
```python
    def compute_turn_score(self):
        """
        Compute a score the last player context.
        Args:
            response: The player response string to be scored.
            context: The context message that was added to the player message history to produce the response.
        Returns:
            1 if the firstlast game rules were followed, 0 otherwise.
        """
        return self.turn_scores[self.current_turn-1]
```

#### _does_game_proceed()
The `_does_game_proceed()` method determines if the episode should continue. Episodes are ended if:
- The required number of rounds has been reached, making the episode successful.
- The move format rule was violated, aborting the episode.
- The letter rule was violated, losing the episode.
Replace the template method definition with the following:
```python
    def _does_game_proceed(self) -> bool:
        """Check if game should proceed."""
        return (self.current_turn < self.n_turns
                and not self.aborted
                and not self.lose)
```

#### compute_episode_score()
This method calculates an overall score for the episode. We sum up the turn scores, divide them by the target number of turns and multiply this by 100 (to fit the required 0-100 score range). Replace the template method definition with the following:
```python
    def compute_episode_score(self):
        """
        Calculate a score for the episode based on successful turns and target number of turns.
        Returns:
            Episode score value in range 0-100.
        """
        turn_score_sum = sum(self.turn_scores)
        success_ratio = turn_score_sum / self.n_turns
        return success_ratio * 100
```

#### _on_after_game()
Finally, we record counts and values that are used later for scoring with the `_on_after_game()` method. Replace the template method definition with the following:

```python
    def _on_after_game(self) -> None:
        """Log variables needed for scoring."""
        # log a message informing that the game was successfully played:
        if not self.aborted and not self.lose:
            action = {'type': 'info', 'content': 'game successful'}
            self.log_event(from_='GM', to='GM', action=action)
        # log a final message saying that the game did come to an end:
        action = {'type': 'info', 'content': 'end game'}
        self.log_event(from_='GM', to='GM', action=action)
        # log firstlast-specific values:
        self.log_key('Played turns', self.current_turn)
        self.log_key('Complete turns', self.complete_turns)
        self.log_key('Turn scores', self.turn_scores)
        # log standard metrics:
        self.log_key(ms.METRIC_ABORTED, self.aborted)
        self.log_key(ms.METRIC_LOSE, self.lose)
        self.log_key(ms.METRIC_REQUEST_COUNT, self.request_count)
        self.log_key(ms.METRIC_REQUEST_COUNT_PARSED, self.parsed_request_count)
        self.log_key(ms.METRIC_REQUEST_COUNT_VIOLATED, self.violated_request_count)
```

### Full example game master
The cell below shows the combined code for the firstlast game master.

In [ ]:
import os.path
from typing import Dict, Tuple, List, Union
from string import ascii_lowercase as letters
import logging

import numpy as np

import clemcore.metrics as ms
from clemcore.clemgame import GameSpec, GameMaster, GameBenchmark, Player, DialogueGameMaster, GameScorer, \
    GameError, ParseError, RuleViolationError
from clemcore.backends import Model

# import the Speaker player class:
from player import Speaker

# initialize logging:
logger = logging.getLogger(__name__)

class FirstLast(DialogueGameMaster):
    """Implement mechanisms for playing FirstLast."""
    def __init__(self, game_name: str, game_path: str, experiment: Dict, player_models: List[Model]):
        super().__init__(game_name, game_path, experiment, player_models)

    def _on_setup(self, **game_instance) -> None:
        """
        Set up the episode (mandatory).
        Args:
            game_instance: The game instance dict.
        """
        self.game_instance: dict = game_instance

        self.n_turns: int = game_instance['n_turns']

        # instantiate both players:
        self.player_a = Speaker(self.player_models[0], 'A', game_instance['first_letter'])
        self.player_b = Speaker(self.player_models[1], 'B', game_instance['first_letter'])

        # add players, including assigning their initial prompts:
        self.add_player(self.player_a, initial_context=game_instance['prompt_player_a'])
        self.add_player(self.player_b, initial_prompt=game_instance['prompt_player_b'])

        # initialise game variables:
        self.current_turn: int = 0
        self.current_letter: str = game_instance['first_letter']
        # log any additional keys that will be relevant for evaluation
        self.log_key('n_turns', game_instance['n_turns'])

        self.correct_response = False
        self.turn_scores = [0] * (self.n_turns + 1)
        self.complete_turns: int = 0

        # initialise common metrics:
        self.request_count: int = 0
        self.parsed_request_count: int = 0
        self.violated_request_count: int = 0

        # initialise attributes that will be used for the evaluation scores
        self.aborted: bool = False
        self.lose: bool = False

    def _parse_response(self, player: Player, response: str) -> Tuple[str, str]:
        """
        Add the response to the other player's message history and check if the response follows the move format rule,
        then split the response and return the first and last word.
        Args:
            player: The player that produced the response.
            response: The response string.
        Returns:
            Tuple of the first and last word of the response.
        Raises:
            ParseError: If the response is missing 'I SAY: '.
        """
        # increase the number of API requests:
        self.request_count += 1

        if player == self.player_a:
            self.set_context_for(self.player_b, response)
        if player == self.player_b:
            self.set_context_for(self.player_a, response)

        # check for move format tag:
        if not response.startswith("I SAY: "):
            raise ParseError()

        # increase the counter of requests that conform to form rules
        self.parsed_request_count += 1
        # log the event that the string was valid (no strange characters)
        action = {'type': 'metadata', 'content': 'move format followed'}
        self.log_event(from_='GM', to='GM', action=action)

        # remove the move format tag and split on whitespace:
        words = response[7:].split()
        return words[0].lower(), words[-1].lower()

    def _on_parse_error(self, error: GameError):
        """Abort the game due to failed parsing."""
        # set the game to be aborted:
        self.aborted = True
        # increase the counter of requests that violate the move format rule:
        self.violated_request_count += 1
        # log the abortion event:
        action = {'type': 'missing tag', 'content': 'abort'}
        self.log_event(from_='GM', to='GM', action=action)

    def _advance_game(self, player: Player, parsed_response: Tuple[str, str]):
        """
        Check if the parsed response follows the game rules. Then advance the game state, preparing the next player's turn.
        Args:
            player: The current player.
            parsed_response: The parsed response, a tuple of the first and last word of the response.
        """
        first_word_correct_letter = parsed_response[0][0] == self.current_letter  # True if the first letter of the first word in the response is correct
        last_word_correct_letter = parsed_response[0][0] == parsed_response[1][0]  # True if the first letters of the first and last word match
        self.correct_response = first_word_correct_letter and last_word_correct_letter
        if not self.correct_response:
            raise RuleViolationError(f'{parsed_response[0]}/{parsed_response[1]} violates rules')  # RuleViolationError inherits from GameError
        else:
            # log the fact that the answer was correct:
            action = {'type': 'valid response',
                      'content': f'{parsed_response[0]}/{parsed_response[1]} conforms to rules'}
            self.log_event(from_='GM', to='GM', action=action)
            # set the current turn's score to 1:
            self.turn_scores[self.current_turn] = 1
        # increment current turn:
        self.current_turn += 1
        # increment complete turns:
        self.complete_turns += 1
        # update the letter being played:
        current_index = letters.index(self.current_letter)
        self.current_letter = letters[current_index + 1]

    def _on_game_error(self, error: GameError):
        """Lose the game due to violated rules."""
        self.lose = True
        # log the fact that the game is now lost:
        action = {'type': 'rule violation',
                  'content': error.reason}
        self.log_event(from_='GM', to='GM', action=action)

    def compute_turn_score(self):
        """
        Compute a score the last player context.
        Args:
            response: The player response string to be scored.
            context: The context message that was added to the player message history to produce the response.
        Returns:
            1 if the firstlast game rules were followed, 0 otherwise.
        """
        return self.turn_scores[self.current_turn-1]

    def _does_game_proceed(self) -> bool:
        """Check if game should proceed."""
        return (self.current_turn < self.n_turns
                and not self.aborted
                and not self.lose)

    def compute_episode_score(self):
        """
        Calculate a score for the episode based on successful turns and target number of turns.
        Returns:
            Episode score value in range 0-100.
        """
        turn_score_sum = sum(self.turn_scores)
        success_ratio = turn_score_sum / self.n_turns
        return success_ratio * 100

    def _on_after_game(self) -> None:
        """Log variables needed for scoring."""
        # log a message informing that the game was successfully played:
        if not self.aborted and not self.lose:
            action = {'type': 'info', 'content': 'game successful'}
            self.log_event(from_='GM', to='GM', action=action)
        # log a final message saying that the game did come to an end:
        action = {'type': 'info', 'content': 'end game'}
        self.log_event(from_='GM', to='GM', action=action)
        # log firstlast-specific values:
        self.log_key('Played turns', self.current_turn)
        self.log_key('Complete turns', self.complete_turns)
        self.log_key('Turn scores', self.turn_scores)
        # log standard metrics:
        self.log_key(ms.METRIC_ABORTED, self.aborted)
        self.log_key(ms.METRIC_LOSE, self.lose)
        self.log_key(ms.METRIC_REQUEST_COUNT, self.request_count)
        self.log_key(ms.METRIC_REQUEST_COUNT_PARSED, self.parsed_request_count)
        self.log_key(ms.METRIC_REQUEST_COUNT_VIOLATED, self.violated_request_count)

# GameScorer implementation
Each clemgame needs a game scorer, a child class of the clemcore `GameScorer` base class. The game scorer reads episode records and calculates evaluation scores. The `GameScorer` base class handles standard scores by default, so we only need to complete two methods: `score_turns()` to calculate and log turn-level scores, and `log_main_score()` to calculate and log the episode's benchmark score. Replace the template definition with the following:

In [ ]:
class FirstLastScorer(GameScorer):
    """Scorer for the firstlast game."""
    def __init__(self, game_name: str, experiment: Dict, game_instance: Dict):
        super().__init__(game_name, experiment, game_instance)

    def score_turns(self, episode_interactions: Dict) -> None:
        """Calculate and log turn-level scores."""
        played_turns = episode_interactions['Played turns']
        turn_scores = episode_interactions['Turn scores']
        for turn in range(0, played_turns):
            self.log_turn_score(turn, "turn score", turn_scores[turn])

    def log_main_score(self, episode_interactions: Dict):
        complete_turns = episode_interactions['Complete turns']
        n_turns = episode_interactions['n_turns']
        aborted = int(episode_interactions[ms.METRIC_ABORTED])
        # IMPORTANT: aborted episodes MUST have a bench score of NaN!
        bench_score = complete_turns / n_turns if not aborted else np.nan
        self.log_episode_score(ms.BENCH_SCORE, bench_score)

# Game registry
Every clemgame needs a game registry entry, stored as `clemgame.json` in the game's root directory. It is required for the clemcore framework to find the game and its file system path.

The contents for firstlast are:
```json
{
  "game_name": "firstlast",
  "description": "Firstlast game between two players that must match the first letters of the first and last word of their responses.",
  "main_game": "firstlast",
  "players": "two",
  "image": "none",
  "languages": ["en"],
  "benchmark": []
}
```
Create a JSON file in the `firstlast` directory and copy the game registry entry into it.

# GameBenchmark implementation

We need to define which classes the `clemcore` framework needs to run our game. This can be done in the same file where the GameMaster lives. We need to create a child of `GameBenchmark`. We also add a short description of the game and the method that calls the FirstLast game master.


In [ ]:
# put this at the end of firstlast/master.py

# always add the GameBenchmark child with this structure
class FirstLastGameBenchmark(GameBenchmark):
    """Integrate the game into the benchmark run."""
    def __init__(self, game_spec: GameSpec):
        super().__init__(game_spec)

    def create_game_master(self,
                           experiment: Dict,
                           player_models: List[Model]
                           ) -> GameMaster:
        return FirstLast(self.game_name, self.game_path, experiment, player_models)

    def create_game_scorer(self, experiment: Dict, game_instance: Dict) -> GameScorer:
        return FirstLastScorer(self.game_name, experiment, game_instance)

With this done, the example `firstlast` clemgame is ready to run - but we should test it, and it is very likely that you need to refine your clemgame. The next section has suggestions how to go about this.
# Testing and refinement
## Running your clemgame
To run the example `firstlast` clemgame, make sure that you have activated the virtual environment that has `clemcore` installed. We will use a model named `gemma-3-27b-it-UP` here, which is accessing a remote API. Enter the following into your terminal:
```
(clemcore_venv) clem run -g firstlast -m gemma-3-27b-it-UP
```
This will run the game with the instances we created earlier, create `interactions.json` and `requests.json` record files in the `results/firstlast` directory in subdirectories for each experiment and episode, and then score episodes based on the records, producing a `scores.json` file for each episode.

## Checking game records
The most important record file of each episode to check is `interactions.json`, as it is supposed to contain all relevant information to score and transcribe the episode.

The `interactions.json` file in the `results/firstlast/0_dogs/episode_0` directory should have the following content:
```json
{
  "meta": {
    "experiment_name": "dogs",
    "game_id": 0,
    "dialogue_pair": "gemma-3-27b-it-UP-t0.0--gemma-3-27b-it-UP-t0.0"
  },
  "players": {
    "GM": "Game master for firstlast",
    "Player 1 (Speaker)": "Player 1 (Speaker) (Speaker): gemma-3-27b-it-UP",
    "Player 2 (Speaker)": "Player 2 (Speaker) (Speaker): gemma-3-27b-it-UP"
  },
  "turns": [
    [
      {
        "from": "GM",
        "to": "Player 1 (Speaker)",
        "timestamp": "2025-04-16T18:57:53.881302",
        "action": {
          "type": "send message",
          "content": "Let's play a game. You must have a conversation about dogs with your partner. Your first turn must start and end with words that begin with the letter a. The reply of your partner must be similar, with the letter that comes after a in the alphabet. Then it's your turn again with the next letter, and so on. You'll do it for 5 turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose.",
          "label": "context"
        }
      },
      {
        "from": "Player 1 (Speaker)",
        "to": "GM",
        "timestamp": "2025-04-16T18:57:55.901125",
        "action": {
          "type": "get message",
          "content": "Okay, I understand the rules! Let's begin.\n\n**I SAY:** Absolutely adorable animals are dogs, aren't they? A fluffy golden retriever is my dream!\n\n\n\n**(Waiting for partner's \"B\" response)**",
          "label": "response"
        }
      },
      {
        "from": "GM",
        "to": "GM",
        "timestamp": "2025-04-16T18:57:55.901125",
        "action": {
          "type": "invalid format",
          "content": "abort"
        }
      },
      {
        "from": "GM",
        "to": "GM",
        "timestamp": "2025-04-16T18:57:55.901125",
        "action": {
          "type": "info",
          "content": "end game"
        }
      }
    ]
  ],
  "n_turns": 5,
  "Played turns": 1,
  "Complete turns": 0,
  "Turn scores": [
    1,
    0,
    0,
    0,
    0,
    0
  ],
  "Aborted": true,
  "Lose": false,
  "Request Count": 1,
  "Parsed Request Count": 0,
  "Violated Request Count": 1
}
```
The `scores.json` file in the `results/firstlast/0_dogs/episode_0` directory should have the following content:
```json
{
  "turn scores": {
    "0": {
      "turn score": 1
    }
  },
  "episode scores": {
    "Aborted": 1,
    "Lose": 0,
    "Success": 0,
    "Request Count": 1,
    "Parsed Request Count": 0,
    "Violated Request Count": 1,
    "Request Success Ratio": 0.0,
    "Main Score": NaN
  }
}
```

As we can see, the episode was aborted (`"episode scores": {"Aborted": 1,...}` in `scores.json`) due to a violated request (`"Violated Request Count": 1`).

Checking `interactions.json`, we can see the episode was aborted in the first turn, due to Player 1's response having invalid format:
```json
      {
        "from": "GM",
        "to": "GM",
        "timestamp": "2025-04-16T18:57:55.901125",
        "action": {
          "type": "invalid format",
          "content": "abort"
        }
      }
```
The `get message` event recorded right above contains the response text that led to this:
```json
      {
        "from": "Player 1 (Speaker)",
        "to": "GM",
        "timestamp": "2025-04-16T18:57:55.901125",
        "action": {
          "type": "get message",
          "content": "Okay, I understand the rules! Let's begin.\n\n**I SAY:** Absolutely adorable animals are dogs, aren't they? A fluffy golden retriever is my dream!\n\n\n\n**(Waiting for partner's \"B\" response)**",
          "label": "response"
        }
      }
```
The response content is `Okay, I understand the rules! Let's begin.\n\n**I SAY:** Absolutely adorable animals are dogs, aren't they? A fluffy golden retriever is my dream!\n\n\n\n**(Waiting for partner's \"B\" response)**` - the model fails to follow the move format rule by producing an acknowledgment of the rules stated in the initial prompt, leading to the abort. (It also does not follow the first letter rule for the last word of its proper reply, but we'll only address the first issue to keep this example short. Fully refining prompts and processing by the game master can be an extended process.)

## Refinement: Initial prompt engineering
Based on the move format violation issue above, we'll adjust the initial prompts, adding the instruction `Do not state that you understood the rules or add that you are waiting for your partner's response.` to them.

Player A:
```Let's play a game. You must have a conversation about $topic with your partner. Your first turn must start and end with words that begin with the letter $letter. The reply of your partner must be similar, with the letter that comes after $letter in the alphabet. Then it's your turn again with the next letter, and so on. You'll do it for $n_turns turns. Always start your utterance with I SAY: and then give your answer. Do not state that you understood the rules or add that you are waiting for your partner's response. If you break the rules, you lose.```

Player B:
```Let's play a game. You must have a conversation about $topic with your partner. Their first turn must start and end with words that begin with the letter $letter. Your reply must be similar, with the letter that comes after $letter in the alphabet. Then it's their turn again with the next letter, and so on. You'll do it for $n_turns turns. Always start your utterance with I SAY: and then give your answer. Do not state that you understood the rules or add that you are waiting for your partner's response. If you break the rules, you lose.```

We change this in the `.template` files, then run `instancegenerator.py` to use these new initial prompts in our instance files. (If your game is more complicated and might require more prompt engineering, it can be useful to load the initial prompt templates in your game master and replace template placeholders in its episode setup - that way, you can skip the instance generation step.)

**IMPORTANT:** Do not be tempted to optimize your prompting for a single model, as clemgames are intended to be run with a large number of models, each likely handling the prompts differently. Always test with multiple models, and keep your prompting general!


Then we run the game again like before. The `interactions.json` file in the `results/firstlast/0_dogs/episode_0` directory should now have the following content:
```json
{
  "meta": {
    "experiment_name": "dogs",
    "game_id": 0,
    "dialogue_pair": "gemma-3-27b-it-UP-t0.0--gemma-3-27b-it-UP-t0.0"
  },
  "players": {
    "GM": "Game master for firstlast",
    "Player 1 (Speaker)": "Player 1 (Speaker) (Speaker): gemma-3-27b-it-UP",
    "Player 2 (Speaker)": "Player 2 (Speaker) (Speaker): gemma-3-27b-it-UP"
  },
  "turns": [
    [
      {
        "from": "GM",
        "to": "Player 1 (Speaker)",
        "timestamp": "2025-04-16T20:21:21.125266",
        "action": {
          "type": "send message",
          "content": "Let's play a game. You must have a conversation about dogs with your partner. Your first turn must start and end with words that begin with the letter a. The reply of your partner must be similar, with the letter that comes after a in the alphabet. Then it's your turn again with the next letter, and so on. You'll do it for 5 turns. Always start your utterance with I SAY: and then give your answer. Do not state that you understood the rules or add that you are waiting for your partner's response. If you break the rules, you lose.",
          "label": "context"
        }
      },
      {
        "from": "Player 1 (Speaker)",
        "to": "GM",
        "timestamp": "2025-04-16T20:21:21.901309",
        "action": {
          "type": "get message",
          "content": "I SAY: Absolutely adorable animals are dogs, aren't they?",
          "label": "response"
        }
      },
      {
        "from": "GM",
        "to": "GM",
        "timestamp": "2025-04-16T20:21:21.901309",
        "action": {
          "type": "metadata",
          "content": "valid string"
        }
      },
      {
        "from": "GM",
        "to": "GM",
        "timestamp": "2025-04-16T20:21:21.901309",
        "action": {
          "type": "parse",
          "content": "absolutely/they? violates rules"
        }
      },
      {
        "from": "GM",
        "to": "GM",
        "timestamp": "2025-04-16T20:21:21.901309",
        "action": {
          "type": "info",
          "content": "end game"
        }
      }
    ]
  ],
  "n_turns": 5,
  "Played turns": 1,
  "Complete turns": 0,
  "Turn scores": [
    1,
    0,
    0,
    0,
    0,
    0
  ],
  "Aborted": false,
  "Lose": true,
  "Request Count": 1,
  "Parsed Request Count": 1,
  "Violated Request Count": 0
}
```
As we can see, the additional instruction worked! The model responded with `I SAY: Absolutely adorable animals are dogs, aren't they?`, without the extraneous acknowledgements of the game procedure. It followed the move format rule as well, correctly starting the response with `I SAY: `. However, the last word it produced is `they?`, which violates the rule that the last word has to start with `a` this turn.

The notable difference here is that this episode was not ABORTED, but LOST, as the move format rule *was* followed, but further game rules were not.

This wraps up this example clemgame implementation. Your own clemgame will very likely be more complicated, so it is recommended to familiarize yourself deeper with the `clemcore` base classes once you are developing a new clemgame.